In [1]:
import torch
import torch_geometric.transforms as T
import matplotlib.pyplot as plt
import collections
import numpy as np
import pandas as pd
import networkx as nx
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import scipy
from scipy.io import loadmat as lm

from torch_geometric.utils.convert import  from_scipy_sparse_matrix

OSError: [WinError 127] Не найдена указанная процедура

Python 


**ПРОБЛЕМЫ ВОПРОСЫ**:
- BlogCatalog, CoCit, Wiki имебют матрицу groupб утвреждается что это лейблы но как быть если у BlogCatalog, Wiki на каждую вершину по более чем одному лейблу -  поставила всем как - фичи
- Epinion, Arxiv не имеют ни лейблов ни фичей

In [ ]:
def to_networkx(data, node_attrs=None, edge_attrs=None):
    r"""Converts a data object graph to a networkx graph.

    Args:
        data (torch_geometric.data.Data): The data object.
        node_attrs (iterable of str, optional): The node attributes to be
            copied. (default: :obj:`None`)
        edge_attrs (iterable of str, optional): The edge attributes to be
            copied. (default: :obj:`None`)
    """

    G = nx.Graph()
    if node_attrs is not None:
        
        for i,attr in enumerate(node_attrs):
            G.add_node(i, feature = str(attr.tolist()))
    else:
        G.add_nodes_from(list(range((data.num_nodes))))
    
    for i, (u, v) in enumerate(data.edge_index.t().tolist()):
        G.add_edge(u, v)
    
    return G


In [ ]:
#тут мульти-лейбл
Blogcatalog = lm('datasets/blogcatalog.mat')
blogcatalog_edgeindex,_=from_scipy_sparse_matrix(Blogcatalog['network'])
#blogcatalog_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Blogcatalog['group']))
data_BlogCatalog = Data(edge_index = blogcatalog_edgeindex)
G_BlogCatalog = to_networkx(data_BlogCatalog)
#nx.from_scipy_\sparse_matrix(data_Blogcatalog['network'])

In [ ]:
CoCit = lm('datasets/CoCit.mat')
CoCit_edgeindex,_=from_scipy_sparse_matrix(CoCit['network'])
CoCit_y_matrix = torch.tensor(scipy.sparse.csr_matrix.toarray(CoCit['group']))
CoCit_y = CoCit_y_matrix.nonzero().t()[1]
data_CoCit = Data(edge_index = CoCit_edgeindex, y = CoCit_y)
G_CoCit=to_networkx(data_CoCit)

In [ ]:
#тут мульти-лейбл
Wiki = lm('datasets/Wiki.mat')
Wiki_edgeindex,_=from_scipy_sparse_matrix(Wiki['network'])
#Wiki_x = torch.tensor(scipy.sparse.csr_matrix.toarray(Wiki['group']))
data_Wiki = Data(edge_index = Wiki_edgeindex)
G_Wiki=to_networkx(data_Wiki)

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Blogcatalog['network']).transpose() == scipy.sparse.csr_matrix.toarray(Blogcatalog['network']))

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(CoCit['network']).transpose() == scipy.sparse.csr_matrix.toarray(CoCit['network']))

In [ ]:
#провекра на симметричность а значит на ненаправленность
np.all(scipy.sparse.csr_matrix.toarray(Wiki['network']).transpose() == scipy.sparse.csr_matrix.toarray(Wiki['network']))

In [ ]:
with open('datasets/soc-Epinions1.txt','r') as f:
    data_Epinion = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Epinion[4:]))
#print(new_list)
dataedge_ep = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set1=set(collections.Counter((np.array(dataedge_ep)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge_ep)).transpose()[0]).keys())
map_ep={}
for j,i in enumerate(set1):
    map_ep[i] = j
dataedge_ep_new = list(map(lambda x: [map_ep[x[0]],map_ep[x[1]]], dataedge_ep))
data_edge_index_Epinion = torch.tensor(dataedge_ep_new).t()
data_Epinion_torch = Data(edge_index=data_edge_index_Epinion)
G_Epinion = to_networkx(data_Epinion_torch)

In [ ]:
with open('datasets/CA-GrQc.txt','r') as f:
    data_Arxiv = f.readlines()

new_list=list(map(lambda x: x.split('\t'),data_Arxiv[4:]))
#print(new_list)
dataedge = list(map(lambda x: [int(x[0]),int(x[1].split('\n')[0])],new_list))
set2=set(collections.Counter((np.array(dataedge)).transpose()[1]).keys()) | set(collections.Counter((np.array(dataedge)).transpose()[0]).keys())
map_ar={}
for j,i in enumerate(set2):
    map_ar[i] = j
    
dataedge_ar_new = list(map(lambda x: [map_ar[x[0]],map_ar[x[1]]], dataedge))

data_edge_index_Arxiv = torch.tensor(dataedge_ar_new).t()
data_Arxiv_torch = Data(edge_index=data_edge_index_Arxiv)
G_Arxiv = to_networkx(data_Arxiv_torch)

In [ ]:
from torch_geometric.datasets import Planetoid

data_Cora =Planetoid(root='/tmp/Cora', name='Cora',transform=T.NormalizeFeatures())[0]
#data_cora =dataset[0]
data_Citeseer =Planetoid(root='/tmp/Citeseer', name='Citeseer',transform=T.NormalizeFeatures())[0]
data_Pubmed =Planetoid(root='/tmp/Pubmed', name='Pubmed',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Amazon

data_Photo =Amazon(root='/tmp/Photo', name='Photo',transform=T.NormalizeFeatures())[0]
data_Computers =Amazon(root='/tmp/Computers', name='Computers',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Coauthor

data_CS =Coauthor(root='/tmp/CS', name='CS',transform=T.NormalizeFeatures())[0]
data_Physics =Coauthor(root='/tmp/Physics', name='Physics',transform=T.NormalizeFeatures())[0]

from torch_geometric.datasets import Reddit


#social networks: Reddit, Flickr
from torch_geometric.datasets import Flickr
data_Flickr =Flickr(root='/tmp/Flickr',transform=T.NormalizeFeatures())[0]
# data_Reddit = Reddit(root='/tmp/Reddit',transform=T.NormalizeFeatures()) очень тяжелый 


from torch_geometric.datasets import PPI

data_PPI = PPI(root='/tmp/PPI', split="train",transform=T.NormalizeFeatures())[0] #тут только train

In [ ]:
datasets = [data_Cora, data_Citeseer, data_Pubmed, data_Photo, data_Computers, data_CS, data_Physics,data_Flickr,data_CoCit,data_BlogCatalog,data_Wiki,data_Arxiv_torch,data_Epinion_torch]
datasets_names = ['Cora', 'Citeseer', 'Pubmed', 'Photo', 'Computers', 'CS', 'Physics','Flickr','CoCit','BlogCatalog','Wikipedia','Arxiv','Epinion']

In [ ]:
graphs = []
for dataset in datasets[:8]:
    graphs.append(to_networkx(dataset,dataset.x))
for dataset in datasets[8:]:
    graphs.append(to_networkx(dataset))


In [ ]:
for i,gr in enumerate(graphs):
    if nx.number_connected_components(graphs[i]) == 1:
        print(i,datasets_names[i])
#2,5,6,7,9,10

In [ ]:
number_of_nodes = []
clustering_coef=[]
density=[]
degree_assort=[]
number_of_features=[]
number_of_labels = []
attribute_assort = []
isdirected=[]
avg_degree = []
avg_path = []
 
    
for i,gr in enumerate(graphs):
        print(i)
        try:
            n_f=datasets[i].num_features
            number_of_features.append(n_f)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_features.append(datasets[i].shape(1))
            else:
                number_of_features.append(0)
        
        try:
            i_d=(datasets[i].is_directed())
            isdirected.append(i_d)
        except:
            if (datasets_names[i] == 'CoCit') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                isdirected.append(False)
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion'):
                isdirected.append(True)
        
        try:
            n_l = len(collections.Counter(datasets[i].y.tolist()).keys())
            number_of_labels.append(n_l)
        except:
            if (datasets_names[i] == 'CoCit'):
                number_of_labels.append(len(collections.Counter(datasets[i].y).keys()))
            elif (datasets_names[i] == 'Arxiv') or (datasets_names[i]=='Epinion') or (datasets_names[i]=='BlogCatalog') or (datasets_names[i]=='Wikipedia'):
                number_of_labels.append(None)
        
        try:
            a_s = nx.attribute_assortativity_coefficient(gr,'feature')
            attribute_assort.append(a_s)
        except:
            attribute_assort.append(None)
        
            
        number_of_nodes.append(gr.number_of_nodes())
            #плотность
        density.append(nx.density(gr))
             #ассортативность
        degree_assort.append(nx.degree_assortativity_coefficient(gr))
        clustering_coef.append(np.mean(list(nx.clustering(gr).values())))
        avg_degree.append(np.mean(np.array(list(dict(graphs[1].degree).values()))))
        
        try:
            ep = nx.average_shortest_path_length(gr)
            avg_path.append(ep)
        except:
            ep = 0
            n = 0
            for component in (nx.connected_components(gr)):
                ep+=nx.average_shortest_path_length(gr.subgraph(component))
                n+=1
            avg_path.append(ep/(n))

In [ ]:
number_of_nodes = []
clustering_coef=[]
density=[]
degree_assort=[]
number_of_features=[]
number_of_labels = []
attribute_assort = []
isdirected=[]
avg_degree = []
avg_path = []

In [ ]:
df = pd.DataFrame({'Name of dataset' : datasets_names,
                                'Number of nodes (NN)' :number_of_nodes,'Number of features (NF)' :number_of_features,'Number of lables (NL)':number_of_labels , 'Directed? ' :isdirected,
                                'Clustering coefficient (CC)': clustering_coef,'Density(D)':density, 'Degree assortativity coefficient(DAC)':degree_assort, 'Attribute assortativity coefficient(AAC)': attribute_assort, 'Average Degree (AD)': avg_degree, 'Average shortest path': avg_path })
df.sort_values(by=['Attribute assortativity coefficient(AAC)']) #добавить атрибуты, направленность

In [ ]:
import pickle 
with open('analytics.pickle','wb') as f:
    pickle.dump(df,f)

In [ ]:
df.sort_values(by=['Pearson degree assortativity coefficient (PDAC)'])

In [ ]:
df.sort_values(by=['Degree assortativity coefficient(DAC)'])

In [ ]:
for dataset in datasets:
    print(datasets[i].x.shape)